# Analyze 2020 solution

## Goal

Analyze the 2020 solution to see how easy is to create an ensemble with it.

## Imports

In [ ]:
import os
import json
import random
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib as mpl
import numpy as np

plt.plot()
plt.close('all')
plt.rcParams["figure.figsize"] = (15, 4)
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['font.size'] = 16

## Code

In [ ]:
def plot_task(task, task_id):
    all_samples = task['train'] + task['test']
    for plot_idx, sample in enumerate(all_samples):
        plt.subplot(1, len(all_samples), plot_idx+1)
        plot_grid(sample['input'])
        if plot_idx < len(task['train']):
            plt.title(f'train {plot_idx}')
        else:
            plt.title(f'test {plot_idx-len(task["train"])}')
    plt.suptitle(f'Inputs for task {task_id}')
    plt.show()
    for plot_idx, sample in enumerate(all_samples):
        plt.subplot(1, len(all_samples), plot_idx+1)
        plot_grid(sample['output'])
        if plot_idx < len(task['train']):
            plt.title(f'train {plot_idx}')
        else:
            plt.title(f'test {plot_idx-len(task["train"])}')
    plt.suptitle(f'Outputs for task {task_id}')
    plt.show()


def plot_grid(grid):
    grid = np.array(grid, dtype=int)
    cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
    norm = colors.Normalize(vmin=0, vmax=9)
    plt.imshow(grid, cmap=cmap, norm=norm)
    plt.grid(True,which='both',color='lightgrey', linewidth=0.5)
    plt.xticks(np.arange(-0.5, grid.shape[1]), [])
    plt.yticks(np.arange(-0.5, grid.shape[0]), [])
    plt.xlim(-0.5, grid.shape[1]-0.5)

    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            plt.text(j, i, grid[i, j], ha='center', va='center')

In [ ]:
def plot_solution(solution, title):
    for plot_idx, sample in enumerate(solution.values()):
        plt.subplot(1, len(solution), plot_idx+1)
        plot_grid(sample)
        plt.title(f'attempt {plot_idx + 1}')
    plt.suptitle(title)
    plt.show()

In [ ]:
def analyze_submission(submission_filepath, solutions_filepath):
    with open(submission_filepath, 'r') as f:
        submission = json.load(f)
    with open(solutions_filepath, 'r') as f:
        solutions = json.load(f)
    remove_repeated_attempts(submission)
    verify_second_attempt_is_empty_if_first_is_empty(submission)
    # remove_repeated_attempts(submission)
    # print('not_empty', measure_not_empty_attempts(submission))
    # print('correct', measure_accuracy(submission, solutions))
    compute_metrics(submission, solutions)

def compute_metrics(submission, solution):
    correct = dict(attempt_1=0, attempt_2=0)
    answered = dict(attempt_1=0, attempt_2=0)
    total = 0
    for task_id, values in submission.items():
        for sample_id, sample in enumerate(values):
            total += 1
            for key in correct.keys():
                if sample[key] == solution[task_id][sample_id]:
                    correct[key] += 1
                if sample[key]:
                    answered[key] += 1
    print(f'Total number of samples: {total}')
    for key in correct.keys():
        print(f'{key} recall: {answered[key] / total:.1%} precision: {correct[key] / answered[key]:.1%} accuracy: {correct[key] / total:.1%}')

def measure_not_empty_attempts(submission):
    accuracy = dict(attempt_1=0, attempt_2=0)
    for task_id, values in submission.items():
        for sample_id, sample in enumerate(values):
            for key in accuracy.keys():
                if sample[key]:
                    accuracy[key] += 1
    return accuracy

def measure_accuracy(submission, solution):
    accuracy = dict(attempt_1=0, attempt_2=0)
    for task_id, values in submission.items():
        for sample_id, sample in enumerate(values):
            for key in accuracy.keys():
                if sample[key] == solution[task_id][sample_id]:
                    accuracy[key] += 1
    return accuracy

def remove_repeated_attempts(submission):
    repetitions = 0
    for task_id, values in submission.items():
        for sample_id, sample in enumerate(values):
            if sample['attempt_1'] and sample['attempt_1'] == sample['attempt_2']:
                repetitions += 1
                sample['attempt_2'] = []
    print(f'There were {repetitions} repeated attempts')

def verify_second_attempt_is_empty_if_first_is_empty(submission):
    errors = 0
    for task_id, values in submission.items():
        for sample_id, sample in enumerate(values):
            if not sample['attempt_1'] and sample['attempt_2']:
                errors += 1
    if errors:
        print(f'There were {errors} errors where the first attempt was empty but the second was not')

## Analysis

In [ ]:
analyze_submission('/mnt/hdd0/Kaggle/arc24/submissions/submission_2020_solution.json',
                   '/mnt/hdd0/Kaggle/arc24/data/arc-agi_training_solutions.json')

In [ ]:
analyze_submission('/mnt/hdd0/Kaggle/arc24/submissions/sub_icecube.json',
                   '/mnt/hdd0/Kaggle/arc24/data/arc-agi_training_solutions.json')

In [ ]:
analyze_submission('/mnt/hdd0/Kaggle/arc24/submissions/program_search_submission.json',
                   '/mnt/hdd0/Kaggle/arc24/data/arc-agi_training_solutions.json')